# Dask dashboard

În cadrul acestei părți o să ne focusăm mai mult pe partea de dashboard din Dask. În cadrul acestei părți nu o să avem de aface cu ce anume au de făcut workerii, ci o să ne focusăm pe ideea de unde putem să vedem asta și cum putem să vedem asta. Pentru început o să creem din nou un client

In [1]:
from dask.distributed import Client

In [2]:
client = Client(n_workers=2, threads_per_worker=1, memory_limit='1GB')

In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 2,Total memory: 1.86 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:49751,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 2
Started: Just now,Total memory: 1.86 GiB
Comm: tcp://127.0.0.1:49760,Total threads: 1
Dashboard: http://127.0.0.1:49763/status,Memory: 0.93 GiB
Nanny: tcp://127.0.0.1:49755,


După ce am creat client-ul și l-am afișat, putem ibserva faptul că există o informație care poartă denumirea de 'Dashboard'. Pentru această informație ne este oferit un link. Acel link putem să îl deschidem în browser. Dacă deschidem acel link, o să ni se afișeze următoarea pagină.

<img src='./SS/02.png'>

In [5]:
import dask.dataframe

In [6]:
ddf = dask.dataframe.read_csv('./data/beer_small.csv', blocksize=10e6)

In [7]:
ddf.map_partitions(type)

Dask Series Structure:
npartitions=2
    object
       ...
       ...
dtype: object
Dask Name: type, 2 graph layers

In [8]:
ddf.map_partitions(type).compute()

0    <class 'pandas.core.frame.DataFrame'>
1    <class 'pandas.core.frame.DataFrame'>
dtype: object

În momentul în care facem anumite operații în dask, atunci acel dashboard o să se modifice, o să primim informații despre computațiile ce au fost făcute în dask, despre workers și despre task-uri. Toate aceste informații pot fi găsite în acel url

<img src='./SS/03.png'>

De exemplu, în cadrul dashboard-ului de 'Profiles' putem observa ce funcții s-au apelat. De exemplu, în imaginea de mai sus putem vedea că pentru linia cu verde acolo s-a apelat metoda 'pandas_read_text'. Metoda aceasta a apelat altă metodă și tot așa, până ce s-a ajuns la rezultatul final. Pentru a putea vedea cum anume funcționează partea de scheduler, putem să utilizăm metoda 'persist()'. Această metodă este într-un fel asemănătoare cu cea de compute, doar că îi spune că păstreze anumite date în memorie, în loc să ne afișeze doar răspunsul, să rețină în partea de scheduler ce anume s-a realizat

In [12]:
ipa = ddf[ddf['beer_style'].str.contains('IPA')]

In [13]:
mean_ipa_review = ipa.groupby('brewery_name').review_overall.agg(['mean', 'count'])

In [14]:
mean_ipa_review.persist()

,mean,count
npartitions=1,,
,float64,int64
,...,...


<img src='./SS/04.png'>

În acest moment, în partea de Graph a dashboard-ului putem observa cum anume funcționează acest dask, cum este distribuit. În partea stângă avem două task-uri separate care se ocupă cu partea de citire de date, iar din moment ce acele task-uri au fost realizate se merge la task-ul unde se face partea de agregare. Poate fi extrem de util pentru a vedea cum anume sunt structurate task-urile. Pentru a putea vedea aceste informații și în dask, putem să utilizăm metoda 'visualize()'

In [16]:
mean_ipa_review.visualize()

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…